## Getting the data from an API to the DataFrame

Retrieving the information from the last 10 ledgers

In [5]:
import requests
import json
import pandas as pd

# Ripple API endpoint
url = "https://s2.ripple.com:51234/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = 90588996

# Define the number of ledgers to retrieve
num_ledgers = 10

# Initialize list to store ledger data
ledger_data = []

# For loop to retrieve information for the last 3 ledgers
for i in range(num_ledgers):
    ledger_index = str(int(latest_ledger_index) - i)  # Move backwards in the ledger sequence
    print(f"Retrieving Ledger Index: {ledger_index}")

    # Create JSON request payload for the ledger
    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    # Send the request
    response = requests.post(url, json=payload)

    # Process the response
    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                ledger_data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("metaData", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")
                })

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)

Retrieving Ledger Index: 90588996
Retrieving Ledger Index: 90588995
Retrieving Ledger Index: 90588994
Retrieving Ledger Index: 90588993
Retrieving Ledger Index: 90588992
Retrieving Ledger Index: 90588991
Retrieving Ledger Index: 90588990
Retrieving Ledger Index: 90588989
Retrieving Ledger Index: 90588988
Retrieving Ledger Index: 90588987


In [6]:
df

,LedgerSequence,CloseTime,TransactionType,Account,Fee,TransactionResult,TakerGets,TakerPays,Issuer,Currency,Sequence,OfferSequence,OwnerCount,Flags
0,90588996,2024-Sep-07 14:05:51.000000000 UTC,OfferCancel,rMffDj93JAQvogvJ2k8196DeAt5HcpdDQC,12,tesSUCCESS,None,None,None,None,76197080,76197076.0,None,0.000000e+00
1,90588996,2024-Sep-07 14:05:51.000000000 UTC,Payment,rajaw3L7R7hgz8F1VVAobVVPnC3Vujvvhb,10,tesSUCCESS,None,None,None,None,89656972,NaN,None,NaN
2,90588996,2024-Sep-07 14:05:51.000000000 UTC,Payment,rwjyDXa3YsdZUJcJSDkyRRvt1qwPfxVNKY,25,tesSUCCESS,None,None,None,None,91310024,NaN,None,2.147484e+09
3,90588996,2024-Sep-07 14:05:51.000000000 UTC,OfferCancel,rJtj42u8QPQWcPiwF3B8sNPb2GMo9gmNub,20,tesSUCCESS,None,None,None,None,90559518,90559517.0,None,0.000000e+00
4,90588996,2024-Sep-07 14:05:51.000000000 UTC,Payment,rapido5rxPmP4YkMZZEeXSHqWefxHEkqv6,10,tesSUCCESS,None,None,None,None,18076201,NaN,None,1.310720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,90588987,2024-Sep-07 14:05:20.000000000 UTC,OfferCreate,rogue5HnPRSszD9CWGSUz8UGHMVwSSKF6,10,tesSUCCESS,{'currency': '42495478000000000000000000000000...,"{'currency': 'XDX', 'issuer': 'rMJAXYsbNzhwp7F...",rMJAXYsbNzhwp7FfYnAsYP5ty3R9XnurPo,XDX,11370226,NaN,None,6.553600e+05
683,90588987,2024-Sep-07 14:05:20.000000000 UTC,OfferCreate,rogue5HnPRSszD9CWGSUz8UGHMVwSSKF6,10,tesSUCCESS,"{'currency': 'XDX', 'issuer': 'rMJAXYsbNzhwp7F...","{'currency': 'OVO', 'issuer': 'r3jQzqfGVagsWNb...",r3jQzqfGVagsWNbSxMJpQV8VK7jHHmdv5j,OVO,11370227,NaN,None,6.553600e+05
684,90588987,2024-Sep-07 14:05:20.000000000 UTC,OfferCreate,rapido5rxPmP4YkMZZEeXSHqWefxHEkqv6,10,tesSUCCESS,"{'currency': 'RLT', 'issuer': 'rUetS7kbVYJZ76z...","{'currency': 'MAG', 'issuer': 'rXmagwMmnFtVet3...",rXmagwMmnFtVet3uL26Q2iwk287SRvVMJ,MAG,18076094,NaN,None,6.553600e+05
685,90588987,2024-Sep-07 14:05:20.000000000 UTC,OfferCreate,rapido5rxPmP4YkMZZEeXSHqWefxHEkqv6,10,tesSUCCESS,{'currency': '42495478000000000000000000000000...,"{'currency': 'RLT', 'issuer': 'rUetS7kbVYJZ76z...",rUetS7kbVYJZ76za5ywa1DgViNZMgT9Bvq,RLT,18076093,NaN,None,6.553600e+05


Showing the raw result for one of the ledgers

In [4]:
import requests
import json

# Definieren Sie den Ripple API-Endpunkt
url = "https://s2.ripple.com:51234/"  # Alternativer Ripple Server

# Ledger-Index, den Sie abfragen möchten
ledger_index = "90588710"  # Verwenden Sie "validated" für den letzten validierten Ledger

# Erstellen der JSON-Anfrage
payload = {
    "method": "ledger",
    "params": [
        {
            "ledger_index": ledger_index,
            "transactions": True,
            "expand": True
        }
    ]
}

# Senden der Anfrage
response = requests.post(url, json=payload)

# Überprüfen der Antwort
try:
    response_json = response.json()  # Versuchen, die Antwort als JSON zu laden
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    print("Fehler: Die Antwort ist kein gültiges JSON.")
    print("Rohantwort vom Server:")
    print(response.text)  # Druckt die tatsächliche Antwort, die vom Server empfangen wurde

{
  "result": {
    "ledger_hash": "0BA0AD93CC3DFA383D044E5261EE510443E69836A326BA34E48EF4E0DEF722A2",
    "ledger_index": 90588710,
    "validated": true,
    "ledger": {
      "account_hash": "42CAA52E085441820BC5CEE134DF633ABD75E1B66AF187863CDF8DF96F6887AC",
      "close_flags": 0,
      "close_time": 779032060,
      "close_time_human": "2024-Sep-07 13:47:40.000000000 UTC",
      "close_time_resolution": 10,
      "close_time_iso": "2024-09-07T13:47:40Z",
      "ledger_hash": "0BA0AD93CC3DFA383D044E5261EE510443E69836A326BA34E48EF4E0DEF722A2",
      "parent_close_time": 779032051,
      "parent_hash": "E6F6F1744AA3C22C548CB6471575DEEE1C81A9621401E226D8A1055A5DEC48FA",
      "total_coins": "99987209508488756",
      "transaction_hash": "F72778273C01FDB119D7A1A2D9FDD5AB030B67D3DEA95C6D8B7D3985710D0B6C",
      "ledger_index": "90588710",
      "closed": true,
      "transactions": [
        {
          "Account": "rsUG78xTs7LXJBpeUVvdkFgDrS3s9as4PK",
          "Amount": "90",
         